In [1]:
!pytest bibliograph/tests.py

============================= test session starts =============================
platform win32 -- Python 3.9.13, pytest-7.1.2, pluggy-1.0.0
rootdir: c:\Users\short\Dropbox\jd\60-69 projects\61 public\61.02 bibliograph
plugins: anyio-3.4.0
collected 42 items

bibliograph\tests.py ..........................................          [100%]

============================= 42 passed in 21.76s =============================


In [ ]:
from crossref.restful import Works, Etiquette
my_etiquette = Etiquette(
    'bibliograph',
    '0.0.1',
    'https://github.com/shortorian/bibliograph',
    'short.devin@gmail.com'
)

works = Works(etiquette=my_etiquette)
result = works.sample(5).select('DOI')
result.do_http_request('get', result.url, only_headers=True, custom_header=result.custom_header)